In [4]:
include("modle.jl")
using .Model
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 

In [2]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 3.61

# 定义铜的FCC晶胞的基矢量
lattice_vectors = (Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))'

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,10,10,10)
fig=visualize_unitcell_atoms(cpcell);

In [3]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return 24*(2/rn^14-1/rn^8)*r
end


interaction = Interaction(lj, Flj, 10.0, 0.1)

x=1:0.001:10
y=interaction.cutenerge.(x)
lines(x,y)

In [4]:
function cell_energeij(cell::UnitCell,interaction::Interaction,i::Int,j::Int;ifnormlize::Bool=true,maxiter::Int=-1,tol::Float64=1e-3)

    cutoff=interaction.cutoff
    atomi=cell.atoms[i]
    atomj=cell.atoms[j]
    a,b,c=cell.copy
    if maxiter==-1
        maxiter=Int(ceil(cutoff/minimum((cell.lattice_vectors*Vector([1,1,1])))))+1
    end
    cni=atomi.cn
    rij=atomj.position-atomi.position
    rij_lt=cell.lattice_vectors*rij

    energe=-interaction.cutenerge(norm(rij_lt))
    for ci in 0:maxiter
        for bi in 0:maxiter
            for ai in 0:maxiter
                rij=atomj.position-atomi.position+ai*Vector([a,0,0])+bi*Vector([0,b,0])+ci*Vector([0,0,c])
                rij_lt=cell.lattice_vectors*rij
                r=norm(rij_lt)
        
                if r>cutoff
                    break
                end
                if r<tol
                    continue
                end
                energe+=interaction.cutenerge(r)
            end
        
        end

    end
    for ci in 0:-1:-maxiter
        for bi in 0:-1:-maxiter
            for ai in 0:-1:-maxiter
                rij=atomj.position-atomi.position+ai*Vector([a,0,0])+bi*Vector([0,b,0])+ci*Vector([0,0,c])
                rij_lt=cell.lattice_vectors*rij
                r=norm(rij_lt)
                if r>cutoff
                    break
                end
                if r<tol
                    continue
                end
                energe+=interaction.cutenerge(r)
            end
        end

    end
    if ifnormlize
        energe=energe/cni
    end
    return energe
end

cell_energeij (generic function with 1 method)

In [5]:
cell_energeij(cpcell,interaction,1,2;ifnormlize=false)

-0.001806427418713431

In [6]:
function cell_energe(cell::UnitCell,interaction::Interaction;ifnormlize::Bool=true,maxiter::Int=-1,tol::Float64=1e-3)
    energe=0.0
    for i in 1:length(cell.atoms)
        for j in 1:length(cell.atoms)
            if i!=j
                energe+=cell_energeij(cell,interaction,i,j,ifnormlize=ifnormlize,maxiter=maxiter,tol=tol)
            end
        end
    end
    return energe
end

cell_energe (generic function with 1 method)

In [19]:
el=Vector{Float64}([])
lattice_constant = 3.61
cl=1.3:0.01:2.5
en=0
for lattice_constant in cl
    # 定义铜的FCC晶胞的基矢量
    lattice_vectors = (Matrix([
        lattice_constant 0.0 0.0; #a1
        0.0 lattice_constant 0.0; #a2
        0.0 0.0 lattice_constant] #a3
    ))
    # 创建铜的原子列表
    atoms = [Atom(pos) for pos in atom_positions]

    cell=UnitCell(lattice_vectors,atoms)
    cpcell=copycell(cell,5,5,5)
    en=cell_energe(cell,interaction,ifnormlize=true)
    push!(el,en)
end


In [20]:
# 创建一个新的 Figure
fig = Figure()

# 创建一个 Axis，并设置坐标范围
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )

lines!(ax,cl,el)
# ylims!(ax, -0.001, 0.001)

fig